In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

In [ ]:
df_train = pd.read_csv("data/train_df.csv")
df_test = pd.read_csv("data/test_df.csv")

**2. Pre-processing**

Because of the different scales between each features, we want to scale our dataset.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
columns_to_scale = ['Factor_A', 'Factor_B', 'Factor_C', 'Factor_D', 'Factor_E', 'Factor_F']
df_train[columns_to_scale] = scaler.fit_transform(df_train[columns_to_scale])
df_test[columns_to_scale] = scaler.transform(df_test[columns_to_scale])

In [ ]:
# df_train['Date'] = pd.to_datetime(df_train['Date'])
# df_test['Date'] = pd.to_datetime(df_test['Date'])

In [ ]:
for df in [df_train, df_test]:
    df['Month'] = df.index.month
    df['Day'] = df.index.day
    df['Weekday'] = df.index.weekday

preparation modelisation

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
target_column = "Electric_Consumption"  
feature_columns_train = [col for col in df_train.columns if col not in ['Date', target_column]] 
feature_columns_test = [col for col in df_test.columns if col not in ['Date']]

In [ ]:
# Préparation des séries temporelles
train_target = df_train[target_column]
train_features = df_train[feature_columns_train]
test_features = df_test[feature_columns_test]

**3. Modelisation**

**3.1 SARIMA**

In [ ]:
# Ajouter les features exogènes (indépendantes) au modèle SARIMA
exog_train = train_features.values
exog_test = test_features.values

# from pmdarima import auto_arima

# # Recherche automatique des meilleurs paramètres SARIMA
# best_model = auto_arima(
#     train_target,
#     exogenous=exog_train,
#     seasonal=True,
#     m=12,  # Période saisonnière
#     start_p=0, max_p=3,  # Intervalle pour p
#     start_q=0, max_q=3,  # Intervalle pour q
#     start_P=0, max_P=2,  # Intervalle pour P
#     start_Q=0, max_Q=2,  # Intervalle pour Q
#     d=None, D=None,  # Détection automatique de d et D
#     trace=True,  # Affiche les résultats pendant la recherche
#     error_action='ignore',  # Ignorer les erreurs pour certaines configurations
#     stepwise=True  # Recherche itérative
# )

# # Résumé du modèle
# print(best_model.summary())

In [ ]:
#configuration modèle SARIMA
sarima_model = SARIMAX(
    endog=train_target,
    exog=exog_train,
    order=(3, 0, 2),            #paramètres ARIMA (p, d, q)
    seasonal_order=(2, 0, 1, 12),  #paramètres saisonniers (P, D, Q, S)
    enforce_stationarity=False,
    enforce_invertibility=False
)

sarima_result = sarima_model.fit(disp=False)

predictions = sarima_result.predict(
    exog=exog_test
)

#ajout predictions dans le dataset
df_test['Electric_Consumption'] = predictions

#visualisation résultats
plt.figure(figsize=(10, 6))
plt.plot(df_train[target_column], label='Train Data', color='blue')
plt.plot(df_test['Electric_Consumption'], label='SARIMA Predictions', color='green')
plt.legend()
plt.title('SARIMA Predictions vs Actual Data')
plt.xlabel('Date')
plt.ylabel('Target')
plt.show()


MissingDataError: exog contains inf or nans

In [ ]:
#on retire "Date" comme index pour pouvoir étudier correctement le dataset avec LSTM ensuite
df_test.reset_index(inplace=True)
df_test[['Date', 'Electric_Consumption']].to_csv("results/predictions.csv", index=False)

**3.2 LSTM**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
df_test = pd.read_csv("data/test_df.csv")
df_test[columns_to_scale] = scaler.transform(df_test[columns_to_scale])
df_test['Date'] = pd.to_datetime(df_test['Date'])
df_test = df_test.sort_values(by='Date')

In [ ]:
target_column = "Electric_Consumption"  
feature_columns_train = [col for col in df_train.columns if col not in ['Date', target_column]] 
feature_columns_test = [col for col in df_test.columns if col not in ['Date']]

In [ ]:
# Définir la variable cible et les features
X_train = df_train[feature_columns_train]  # Toutes les colonnes avant 'target'
y_train = df_train[target_column]  # Colonne 'target'
X_test = df_test[feature_columns_test]

In [ ]:
# Fonction pour créer des séquences
print("Shape initiale de X_train :", X_train.shape)
print("Shape initiale de X_test :", X_test.shape)
def create_sequences(X, y, time_steps):
    if len(X) <= time_steps:
        raise ValueError("Le paramètre `time_steps` est trop grand pour la longueur des données.")
    
    X_seq, y_seq = [], []
    for i in range(len(X) - time_steps):
        X_seq.append(X[i:i + time_steps])
        y_seq.append(y[i + time_steps])
    return np.array(X_seq), np.array(y_seq)


# Paramètre de séquencement (nombre de pas de temps)
time_steps = 10

# Créer les séquences
X_train, y_train = create_sequences(X_train, y_train, time_steps)
X_seq = []
for i in range(len(X_test) - time_steps):
        X_seq.append(X_test[i:i + time_steps])
X_test = np.array(X_seq)

print("Shape finale de X_train :", X_train.shape)
print("Shape finale de X_test :", X_test.shape)

In [ ]:
# Construire le modèle LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(units=1))  # Une sortie pour la prédiction de la target

# Compiler le modèle
model.compile(optimizer='adam', loss='mean_squared_error')

# Entraîner le modèle
model.fit(X_train, y_train, epochs=20, batch_size=32)

In [ ]:
# Prédire sur l'ensemble de test
predicted = model.predict(X_test)

In [ ]:
import numpy as np

# Si `predicted` est à 2 dimensions (par exemple, shape (2150, 1))
if len(predicted) < len(df_test):
    # Créer un padding de même forme que les prédictions
    padding = np.full((len(df_test) - len(predicted), predicted.shape[1]), np.nan)  # Padding avec NaN
    predicted = np.vstack([padding, predicted])  # Empiler verticalement

# Vérification de la taille finale
assert len(predicted) == len(df_test), "Les tailles de `predicted` et `df_test` ne correspondent toujours pas."

# Ajouter les prédictions au DataFrame
df_test['Electric_Consumption'] = predicted.squeeze()  # Retirer les dimensions inutiles si nécessaire

# Remplir les valeurs manquantes ou non valides
df_test['Electric_Consumption'] = df_test['Electric_Consumption'].fillna(0)  # Remplacer NaN par 0 ou une autre valeur

# Exportation au format CSV
df_test[['Date', 'Electric_Consumption']].to_csv("results/predictions2.csv", index=False)
print("Exportation réussie.")